# 向量数据库记忆测试
LangChain本地服务器模拟虚拟男友聊天接口，使用向量数据库做聊天信息保存和检索，本notebook将测试虚拟男友的记忆能力。

测试分3个部分，1）`MemoryMaker`将捏造`UserProxyAgent`提供的玩家基本信息字段。2）`MemoryMaker`生成的记忆将由`MemoryInjector`调用虚拟男友接口注入记忆。3）`MemoryTester`将根据`UserProxyAgent`的提供的基本信息字段询问虚拟男友接口，并测试是否符合`MemoryMaker`生成的记忆输出表格。
## 环境配置



In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location="../",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

## 捏造记忆
`MemoryMaker`将捏造`UserProxyAgent`提供的玩家基本信息字段。

In [ ]:
import autogen
import requests

llm_config = {
    "timeout": 60,
    "seed": 60,
    "config_list": config_list,
    "temperature": 0.5,
}

user_proxy = autogen.UserProxyAgent(
    name="UserProxy",
    system_message="出题者",
    code_execution_config={"last_n_messages": 2, "work_dir": "memory_test"},
    human_input_mode="NEVER"
)

memory_maker = autogen.AssistantAgent(
    name="MemoryMaker",
    system_message="当完成任务后就不处理信息",
    llm_config=llm_config
)

# 记忆注入
conversation_id="sds11180005_1"
system_args="10_10_10_10_10_10_10_10_10"
chat_endpoint="http://localhost:8000/game/chat"
# 创建调用虚拟男友接口的Agent
class MemoryInjector(autogen.AssistantAgent):
    def __init__(self, conversation_id:str, system_args:str, endpoint:str, **kwargs):
        super().__init__(**kwargs)
        self._reply_func_list = []
        self.register_reply(autogen.Agent, reply_func=MemoryInjector.redirect_chat_endpoint, config={"conversation_id":conversation_id, "system_args": system_args, "chat_endpoint": endpoint})

    def redirect_chat_endpoint(self, messages=None, sender=None, config=None):
        conversation_id=config["conversation_id"]
        system_args=config["system_args"]
        headers={
            "characterInfo":conversation_id,
            "systemPromptArgs":system_args
        }
        endpoint=config["chat_endpoint"]
        if messages is None:
            messages = self._oai_messages[sender]

        user_question = messages[-1]["content"]
        response = requests.post(endpoint, headers=headers, json={'Chat':user_question, 'Type':0})
        # print("虚拟男友接口有回复："+response.text)
        return True, response.text

virtual_character = autogen.AssistantAgent(
    name="VirtualCharacter",
    system_message="接受UserProxy结果{makeup}，扮演虚拟人物和MemoryInjector对话，需要在10论对话内将makup的信息表达给MemoryInjector,当makeup信息都表达完后结束对话，记住你是扮演角色",
    llm_config=llm_config,
)

memory_injector = MemoryInjector(
    name="MemoryInjector",
    system_message="接受VirtualCharacter的消息",
    conversation_id=conversation_id,
    system_args=system_args,
    endpoint=chat_endpoint
)

# messages = user_proxy.chat_messages
# print(messages)
# questioner.initiate_chat(
#     memory_injector, 
#     message=messages
# )

# define group chat
groupchat = autogen.GroupChat(agents=[user_proxy, memory_maker, virtual_character, memory_injector], messages=[], max_round=30)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(
    manager, 
    message='问题发给MemoryMaker捏造一个虚构人物的以下信息：职业，参与产业，任职公司名，专业技能，工作内容。并输出以下元素格式的json数组：{"topic":填入上面需要捏造信息的类型, "memory":填入捏造的内容}，最后答案放在makeup:[]内'
)

## 记忆注入
`MemoryMaker`生成的记忆将由`MemoryInjector`调用虚拟男友接口注入记忆。